In the name of God
# AI COMPUTER ASSIGNMENT #1
*Mobina Haghizadeh 810100127*

```First Part:``` 


We load the data and check its general features :

In [757]:
import pandas as pd
data =  pd.read_csv("snacks.csv")

In [758]:
data.head(10)

,Snack,Available Weight,Value
0,MazMaz,10,10
1,Doogh-e-Abali,15,10
2,Nani,5,5
3,Jooj,7,15
4,Hot-Dog,20,15
5,Chips,8,6
6,Nooshaba,12,8
7,Shokolat,6,7
8,Chocoroll,9,12
9,Cookies,11,11


In [759]:
data.describe()

,Available Weight,Value
count,19.000000,19.000000
mean,9.684211,9.473684
std,4.497563,3.168744
min,3.000000,4.000000
25%,6.500000,7.000000
50%,9.000000,10.000000
75%,12.500000,11.500000
max,20.000000,15.000000


We have some constant numbers. So I defined a Constants dataclass to pass it to my functions.
///////constants = Constants(1000, 300, 0.4, 0.1, 70)

In [760]:
from dataclasses import dataclass
import numpy as np

SNACKSLEN = len(data)

MAXWEIGHT = 10.0
MINVALUE = 12
ACCEPTABLERANGE = (2, 4)

@dataclass
class HyperParams:
    populationLen: int
    maxGenerations: int
    carrySize: int
    mutationProbability: float
    crossoverProbability: float
    

Gene:
In the fractional knapsack problem, a gene can represent the fraction or percentage of a food item to be included in the knapsack. It can be encoded as a floating-point value between 0 and 1, indicating the proportion of the food item's weight to be selected.

Chromosome:
A chromosome represents a combination of selected food items and their corresponding fractions. It can be encoded as a list of genes, where each gene corresponds to a food item and represents the fraction of that item to be included.

In [761]:
import random
import warnings

def normalizeChromosome(chromosome):
    chromosome[chromosome < chromosome.sum()/(SNACKSLEN*10)] = 0
    return chromosome / chromosome.sum()

def geneRandomChromosome():
    chromosome = np.random.rand(SNACKSLEN)
    return normalizeChromosome(chromosome).reshape((1,SNACKSLEN))


```Second part:```
To make an initial population, I simply generate random chromosomes with the size of the population.

In [762]:
def initPopulation(populationLen):
    return np.array([geneRandomChromosome() for i in range(populationLen)]).reshape((populationLen, SNACKSLEN))

```Third part:```
I define fitness function as below:
$$ penalty(chromosome) = -1 $$
$$ fitness(chromosome) = sum(gene * value_i) - penalty(chromosome) * value_i $$ 

In [763]:
def fitnessCalculation(chromosome):
    foodsValue = data["Value"].to_numpy().reshape(1, SNACKSLEN)
    foodsWeight = data["Available Weight"].to_numpy().reshape(1, SNACKSLEN)
    totalValue = np.multiply(foodsValue,chromosome).sum()
    totalWeight = np.multiply(foodsWeight,chromosome).sum() 
    
    fitness = totalValue
    
    if totalWeight > MAXWEIGHT:
        # penalty = 0  # Adjust this factor as needed
        # totalValue *= penalty
        fitness = 0
    
    
    return fitness
    

In [764]:
fitnessCalculation(geneRandomChromosome())

9.712400204622513

```Fourth part:```
I have implemented uniform crossover for the crossover operation. In this method, for each gene, we randomly decide whether to assign the corresponding gene from the first parent to the first child or from the second parent. The second child is then created by taking the inverse of the first child's genes.

In [765]:
def uniformCrossover(parents, HyperParams):
    
    if np.random.random() > HyperParams.crossoverProbability:
        return parents
    
    parent1, parent2 = parents
    child1, child2 = np.empty(shape = (SNACKSLEN)), np.empty(shape = (SNACKSLEN))
    
    for i in range(len(parent1)):
        if np.random.bit_generator.randbits(1):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        else:
            child1[i] = parent2[i]
            child2[i] = parent1[i]
            
    return normalizeChromosome(child1), normalizeChromosome(child2)

For the mutation operation, I randomly select a gene from the chromosome and assign it a completely random value. Afterward, I normalize the resulting chromosome to ensure that the gene values are within the desired range or constraints.

In [766]:
def mutation(chromosome, p_mutation):
    
    if np.random.random() > p_mutation:
        return chromosome
    
    index = np.random.randint(SNACKSLEN)
    chromosome[index] = np.random.random()

    return normalizeChromosome(chromosome)

To create the mating pool, I employ rank-based selection. Within the main algorithm function (findOptimal), for each generation, I calculate the fitness of each chromosome. I then check if we have found a solution or not. Next, I select some of the top-performing chromosomes directly to proceed to the next generation. Subsequently, I construct a new mating pool and perform crossover operations on the selected individuals. Finally, I apply mutation to the new population.

In [767]:
valueColumn = data["Value"].to_numpy().reshape((1,SNACKSLEN))
weightColumn = data["Available Weight"].to_numpy().reshape((1,SNACKSLEN))

The purpose of the isSolution method is to determine whether a given chromosome represents a valid solution to a problem. In this case, it is checking if the chromosome represents a valid combination of snacks to be taken in a knapsack.

In [768]:
def isSolution(chromosome):
    takenSnacks = np.count_nonzero(chromosome)
    totalValues = np.multiply(chromosome, valueColumn).sum()
    totalWeights = np.multiply(chromosome, weightColumn).sum()
    return (totalWeights <= 10 and totalValues >= 12 and ACCEPTABLERANGE[0] <= takenSnacks <= ACCEPTABLERANGE[1])

The purpose of the createMatingPool method is to generate a mating pool from a given population of individuals. The mating pool is a subset of the population that is used for the crossover operation in a genetic algorithm.

In [769]:
def createMatingPool(population, populationLen, matingPoolLen, probability):
    
    probability = probability / probability.sum()
    
    selections = np.random.choice(populationLen, matingPoolLen, p = probability)
    
    unique, counts = np.unique(selections, return_counts=True)
    
    return population[selections, :].reshape((-1, 2, SNACKSLEN))

The purpose of the optimalSolution method is to search for an optimal solution using a genetic algorithm. The method initializes a population, evaluates the fitness of each individual, performs selection, crossover, and mutation operations, and iterates through multiple generations until either an optimal solution is found or the maximum number of generations is reached.

In [770]:
       
def optimalSolution(HyperParams):
    
    population = initPopulation(HyperParams.populationLen)
    generationNum = 0
    while generationNum < HyperParams.maxGenerations:
        
        fitnesses = np.apply_along_axis(fitnessCalculation, axis = 1, arr = population)
        sortedIndices = np.argsort(fitnesses)
        population = population[sortedIndices]
        
        isThisSolution = np.apply_along_axis(isSolution, axis = 1, arr = population)
        
        if np.count_nonzero(isThisSolution):
            return population[np.nonzero(isThisSolution)[0],:], generationNum
        
        weights = np.array([i for i in range(1, HyperParams.populationLen + 1)])
        carryPopulation = population[-HyperParams.carrySize:]
        mating_pool = createMatingPool(population, HyperParams.populationLen, HyperParams.populationLen - HyperParams.carrySize, weights)
        
        crossoverPopulation = np.array([uniformCrossover(parents, HyperParams) for parents in mating_pool]).reshape((-1, SNACKSLEN))
        crossoverPopulation = np.apply_along_axis(lambda x: mutation(x, HyperParams.mutationProbability), axis = 1, arr = crossoverPopulation).reshape((-1, SNACKSLEN))
        
        population = np.concatenate((crossoverPopulation, carryPopulation))
        generationNum += 1
        
    return None, HyperParams.maxGenerations

```Fifth part:```
I have written a wrapper function called runGenetic to execute the find_optimal_solution function multiple times and calculate the average number of generations.

In [771]:
def runGenetic(HyperParams):
    
    sumOfGenerationsNum = 0
    
    REPEAT = 5
    found_answer = 0
    
    for i in range(REPEAT):
  
        result, generationsNum = optimalSolution(HyperParams)
        if type(result) == type(None):
            print("Couldn't find answer")
            return 
   
       
        sumOfGenerationsNum += generationsNum
    
    print("Total Value:", np.multiply(result, valueColumn).sum())
    print("Total Weight:", np.multiply(result, weightColumn).sum())

In [772]:
hyperParams = HyperParams(300, 1000, 70, 0.1, 0.4)
runGenetic(hyperParams)

Total Value: 38.064536149951216
Total Weight: 28.235015942505


```Sixth part:```
Testing with different numbers.

In [773]:
hyperParams = HyperParams(500, 1200, 100, 0.2, 0.6)
runGenetic(hyperParams)

Total Value: 13.33516034506877
Total Weight: 8.78213742236056


In [774]:
hyperParams = HyperParams(200, 800, 30, 0.5, 0.7)
runGenetic(hyperParams)

Total Value: 14.62277558931708
Total Weight: 7.329807611037814


In [775]:
hyperParams = HyperParams(400, 500, 60, 0.8, 0.2)
runGenetic(hyperParams)

Total Value: 12.03801897567875
Total Weight: 9.469545977309215


In [778]:
hyperParams = HyperParams(700, 700, 20, 0.2, 0.4)
runGenetic(hyperParams)

Total Value: 14.883876655704661
Total Weight: 7.2802914295508705


# Questions

## ``Question 1``
If the population size is too small, the algorithm may struggle to effectively explore the search space, potentially leading to convergence on suboptimal solutions. Additionally, a small population size can result in decreased genetic diversity, increasing the risk of losing crucial information.
On the other hand, if the population size is too large, it requires more computational resources to process and maintain the population. This increased resource demand can be inefficient and unnecessary. Furthermore, a larger population size may introduce redundant genes, which can hinder the algorithm's ability to converge on optimal solutions.

## ``Question 2``
If the population grows with each generation, it can lead to increased diversity, which can potentially benefit the algorithm by exploring a larger portion of the search space. However, this approach may also require more time and memory resources to handle the larger population size.
On the other hand, keeping the population size constant can help promote the convergence of chromosomes towards an optimal solution. Increasing the population size may introduce additional individuals that can disrupt the convergence process, potentially hindering the algorithm's ability to reach the optimal answer.

## ``Question 3``
Crossover is a mechanism in genetic algorithms that combines genetic material from two parent chromosomes to produce offspring with potentially improved characteristics. On the other hand, mutation involves making direct changes to the genes of a chromosome, which can introduce new genetic variations into the population and help avoid getting stuck in local optima.
For instance, if we monitor the best fitness value in each generation, we may observe that it tends to converge to a certain value. However, the introduction of a mutation can cause the fitness value to deviate from that value and suddenly increase, allowing the algorithm to explore new areas of the search space. This highlights the importance of incorporating mutation in addition to crossover, as relying solely on crossover may result in slower convergence (especially when using uniform crossover).
Conversely, relying solely on mutation is generally not sufficient to reach an optimal solution, as it lacks the ability to combine and recombine genetic material like crossover does. However, there may be specific cases where using only mutation can lead to a satisfactory solution, depending on the problem and the specific constants involved. Nonetheless, in most scenarios, a combination of both crossover and mutation is necessary to achieve an optimal solution in a genetic algorithm.

## ``Question 4``
In the testing phase, we can experiment with different parameter values to optimize the performance of the algorithm and find the answer as quickly as possible. By adjusting parameters such as population size, mutation probability, crossover method, and selection mechanism, we can fine-tune the algorithm to improve its efficiency and effectiveness in solving the problem at hand. Through iterative testing and evaluation, we can identify the parameter values that yield the fastest convergence and highest-quality solutions.

## ``Question 5``
In genetic algorithms, it is often challenging to ensure that the algorithm reaches the global maximum instead of getting stuck at a local maximum. However, we can address this issue through the use of mutation, which introduces random changes to the genetic material. By incorporating mutation, the algorithm gains the ability to explore new areas of the search space, enabling it to break free from local optima and potentially reach the global maximum.
Additionally, to prevent the algorithm from running indefinitely without converging, we can set a limit on the number of generations. By specifying a maximum number of generations, we can terminate the algorithm if it fails to make substantial progress towards the optimal solution within that limit.

## ``Question 6``
By setting a predetermined maximum value for the number of generations, we can control the termination of the algorithm. Once the algorithm reaches this limit, we can stop its execution. At that point, we can retrieve and return the chromosome from the population that has the highest fitness value, indicating the best solution found throughout the evolutionary process. This approach allows us to obtain the most optimal chromosome available within the given generation limit.